# Multiple Task Pneumonia & TB

In [1]:
import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3999)])
 

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
print(('Is your GPU available for use?\n{0}').format(
    'Yes, your GPU is available: True' if tf.test.is_gpu_available() == True else 'No, your GPU is NOT available: False'
))

print(('\nYour devices that are available:\n{0}').format(
    [device.name for device in tf.config.experimental.list_physical_devices()]
))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is your GPU available for use?
Yes, your GPU is available: True

Your devices that are available:
['/physical_device:CPU:0', '/physical_device:XLA_CPU:0', '/physical_device:GPU:0', '/physical_device:XLA_GPU:0']


Load and prepare data

In [3]:
import cv2 as cv
import pickle
import random
import numpy as np
 

In [4]:
tb1_data = pickle.load( open( "C:/Users/uprz2/Downloads/cs330/data/tb1-256-grayscale", "rb" ) )

p_data = pickle.load( open( "C:/Users/uprz2/Downloads/cs330/data/pneumonia-256-grayscale", "rb" ) )
 
# duplicate to have equal amounts
for i in range(3):
    tb1_data = tb1_data + tb1_data

x_data = []
y_data_tb1 = []
y_data_p = []

for v in tb1_data:
    x_data.append( v[0] )
    y_data_tb1.append(v[1])
    y_data_p.append(0)
    

for v in p_data:
    x_data.append( v[0] )
    y_data_tb1.append(0)
    y_data_p.append(v[1])

x_data = np.array(x_data)
y_data_tb1 = np.array(y_data_tb1)
y_data_p = np.array(y_data_p)

s = np.arange(len(x_data))
np.random.shuffle(s)

x_data = x_data[s]
y_data_tb1 = y_data_tb1[s]
y_data_p = y_data_p[s]


fraction_train = 0.8
train_size = int(len(x_data) * fraction_train)

x_train = x_data[0: train_size]/ 255.0
y_train_tb1 = y_data_tb1[0: train_size]
y_train_p = y_data_p[0: train_size]



x_test = np.array(x_data[train_size:]) / 255.0
y_test_tb1 = np.array(y_data_tb1[train_size:])
y_test_p = np.array(y_data_p[train_size:])



# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")


print("Train size: {}, test size: {}".format(len(x_train), len(x_test)))

print("tb1_data size: {}, p_data size: {}".format(len(tb1_data), len(p_data)))


Train size: 8921, test size: 2231
tb1_data size: 5296, p_data size: 5856


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input


tf.keras.backend.clear_session()

model_input = Input(shape=(256, 256, 1))
shared = Conv2D(32, 3, activation='relu')(model_input)
shared = Flatten()(shared)
y1 = Dense(64, activation='relu')(shared)
y1 = Dense(2, name='tb1')(y1)

y2 = Dense(64, activation='relu')(shared)
y2 = Dense(2, name='pneumonia')(y2)

model = Model(inputs=model_input, outputs=[y1, y2])

loss_list =[tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)]
test_metrics = {'tb1': 'accuracy', 'pneumonia': 'accuracy' }
model.compile(loss=loss_list,optimizer='adam',metrics=test_metrics)


model.fit(x_train, [y_train_tb1, y_train_p], batch_size=8, epochs=30)



print("Evaluation: ")
print(model.metrics_names)

model.evaluate(x_test, [y_test_tb1, y_test_p], batch_size=8)


                                

Epoch 1/30
1116/1116 [==============================] - 110s 98ms/step - pneumonia_loss: 0.3531 - tb1_loss: 0.5931 - pneumonia_accuracy: 0.9315 - tb1_accuracy: 0.8561 - loss: 0.9462
Epoch 2/30
1116/1116 [==============================] - 109s 98ms/step - pneumonia_loss: 0.0751 - tb1_loss: 0.1746 - pneumonia_accuracy: 0.9751 - tb1_accuracy: 0.9409 - loss: 0.2497
Epoch 3/30
1116/1116 [==============================] - 110s 99ms/step - pneumonia_loss: 0.0518 - tb1_loss: 0.1142 - pneumonia_accuracy: 0.9812 - tb1_accuracy: 0.9692 - loss: 0.1660
Epoch 4/30
1116/1116 [==============================] - 110s 99ms/step - pneumonia_loss: 0.0234 - tb1_loss: 0.0570 - pneumonia_accuracy: 0.9932 - tb1_accuracy: 0.9890 - loss: 0.0804
Epoch 5/30
1116/1116 [==============================] - 110s 99ms/step - pneumonia_loss: 0.0466 - tb1_loss: 0.0632 - pneumonia_accuracy: 0.9888 - tb1_accuracy: 0.9837 - loss: 0.1099
Epoch 6/30
1116/1116 [==============================] - 110s 99ms/step - pneumonia_loss: 0

[0.1988062709569931,
 0.0035671070218086243,
 0.1952391415834427,
 0.9982070922851562,
 0.9713132977485657]